## 关于DPO
[DPO的数学原理推导](https://www.cnblogs.com/GreenOrange/p/18798910)<br>
如果看不太懂可以结合B站的BV1rixye7ET6 [PPO原理的学习]  以及这个UP的DPO视频讲解来学习<br>
DPO,PPO,GRPO 其实都只是作为一种对齐手段，用RL的方式offline-RL 简单的学会用户的偏好格式，其本质上是用少量的样本+更换loss 来学会对原有SFT模型进行的fine-tune，带来的仅仅只有少量的知识，大部分是挖掘基模的知识。<br>
简而言之，DPO相比于PPO和GRPO，其主要优势在于不需要进行state-value模型的训练，也不需要打reward分的critor模型的训练，仅仅只需要一次训练，即可实现RLHF<br>
目前在huggingface的trl库已经有了PPO和DPO等的实现，可以参考这个库来进行学习

### 设置训练参数


In [1]:
import torch
class sft_args:
    out_dir = "../out"
    epochs = 1
    batch_size = 32
    learning_rate = 5e-4
    device = "cuda:3" if torch.cuda.is_available() else "cpu"
    dtype = "bfloat16"
    use_wandb = False
    wandb_project = "MiniMind-DPO-512"
    num_workers = 1
    ddp = False
    accumulation_steps = 8
    grad_clip = 1.0
    warmup_iters = 0
    log_interval = 100
    save_interval = 100
    local_rank = -1
    embed_dim = 512
    block_num = 8
    max_seqlen = 1024
    use_moe = False
    data_path = "../data/dpo_data.jsonl"  # toy_dataset  
    # data_path = "../data/dpo.jsonl" #full_dataset

### 初始化model

In [2]:
import sys
import os

# 获取当前 notebook 所在目录（trainer/）
current_dir = os.path.dirname(os.path.abspath("__file__"))  # 注意 Jupyter 中可能需要调整
# 或者直接写死路径
current_dir = "/data/zyp/jinbu/ZZY/minimind-v-learn/trainer"

# 上一级目录就是项目根目录，拼接 model 路径
model_dir = os.path.join(os.path.dirname(current_dir), "model")
sys.path.append(model_dir)

# 现在可以用绝对导入
from model import MinimindForCausalLM, MinimindConfig
train_args = sft_args()
train_args.save_dir = os.path.join(train_args.out_dir)
# 确保输出目录存在
os.makedirs(train_args.save_dir, exist_ok=True)
# 初始化模型配置
config = MinimindConfig(
    embed_dim=train_args.embed_dim,
    block_num=train_args.block_num,
    max_seqlen=train_args.max_seqlen,
)
print(f'查看工作设备 {train_args.device}')

/data/zyp/miniconda3/envs/minimind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


查看工作设备 cuda:3


### 加载model

In [3]:
from transformers import AutoTokenizer
import math
from torch.utils.data import DataLoader
import sys
from pathlib import Path

# 项目根目录：/data/zyp/jinbu/ZZY/minimind-v-learn
root_dir = Path("/data/zyp/jinbu/ZZY/minimind-v-learn")

# 将根目录添加到 Python 可搜索路径
sys.path.append(str(root_dir))
from dataset.lm_dataset import PretrainDataset,SFTDataset,DPODataset

def Logger(content):
    print(content)

def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('../model/')
    model = MinimindForCausalLM(lm_config).to(train_args.device)
    ref_model = MinimindForCausalLM(lm_config)
    ref_model.eval()
    ref_model.to(train_args.device)

    moe_path = '_moe' if train_args.use_moe else ''
    ckp = f'{train_args.save_dir}/sft_full_{lm_config.embed_dim}{moe_path}.pth'
    state_dict = torch.load(ckp, map_location=train_args.device)
    model.load_state_dict(state_dict, strict=False)
    ref_model.load_state_dict(state_dict, strict=False)
    Logger(f'加载模型参数 {ckp}')
    Logger(f'LLM可训练总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    return model,ref_model,tokenizer


model, ref_model,tokenizer = init_model(config)
train_ds = DPODataset(
    data_path=train_args.data_path,
    tokenizer=tokenizer,
    max_seqlen=train_args.max_seqlen,
)   
train_loader = DataLoader(
    train_ds,
    batch_size=train_args.batch_size,
    shuffle=False,
    num_workers=train_args.num_workers,
    pin_memory=True,
    drop_last=False
)

loader = iter(train_loader)
batch = next(loader)
# print(f'打印一个 batch 的数据:\nX: {X}\nY: {Y}\nloss_mask: {loss_mask}\n')
# print(f'打印一个 iter 的数据:\n{next(loader)}\n')
# print(f'数据集大小：{len(train_ds)}, DataLoader 大小：{len(loader)}')
print(f"batch = {batch}")

加载模型参数 ../out/sft_full_512.pth
LLM可训练总参数量：38.075 百万


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


batch = {'x_chosen': tensor([[  2,  95, 101,  ...,   3,   3,   3],
        [  2,  95, 101,  ...,   3,   3,   3]]), 'y_chosen': tensor([[ 95, 101, 315,  ...,   3,   3,   3],
        [ 95, 101, 315,  ...,   3,   3,   3]]), 'mask_chosen': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'x_rejected': tensor([[  2,  95, 101,  ...,   3,   3,   3],
        [  2,  95, 101,  ...,   3,   3,   3]]), 'y_rejected': tensor([[ 95, 101, 315,  ...,   3,   3,   3],
        [ 95, 101, 315,  ...,   3,   3,   3]]), 'mask_rejected': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])}


### 学习率和优化器选择

In [4]:
# 优化器方面 选择 AdamW 优化器 并在混精度场景下创建 scaler 进行梯度缩放避免数值下溢
from torch import optim
from contextlib import nullcontext
scaler = torch.cuda.amp.GradScaler(enabled=(train_args.dtype in ['float16', 'bfloat16']))
optimizer = optim.AdamW(model.parameters(), lr=train_args.learning_rate)

device_type = "cuda" if "cuda" in train_args.device else "cpu"
ctx = nullcontext() if device_type == "cpu" else torch.cuda.amp.autocast() # 在 cuda 上启动混精度训练，否则空白上下文

### 设置训练的DPOLoss

In [5]:
from torch.nn import functional as F
def logits_to_probs(logits, labels):
    # logits shape: (batch_size, seq_len, vocab_size)
    # labels shape: (batch_size, seq_len)
    # probs shape: (batch_size, seq_len)
    log_probs = F.log_softmax(logits, dim=2)
    probs = torch.gather(log_probs, dim=2, index=labels.unsqueeze(2)).squeeze(-1)
    return probs


def dpo_loss(ref_probs, probs, beta):
    # ref_probs 和 probs 都是 shape: (batch_size, seq_len)
    # 计算每个样本的平均概率
    ref_probs = ref_probs.mean(dim=1)
    probs = probs.mean(dim=1)

    # 将 chosen 和 rejected 数据分开
    batch_size = ref_probs.shape[0]
    chosen_ref_probs = ref_probs[:batch_size // 2]
    reject_ref_probs = ref_probs[batch_size // 2:]
    chosen_probs = probs[:batch_size // 2]
    reject_probs = probs[batch_size // 2:]

    # 计算对数比率，比较偏好差异
    pi_logratios = chosen_probs - reject_probs
    ref_logratios = chosen_ref_probs - reject_ref_probs
    logits = pi_logratios - ref_logratios
    loss = -F.logsigmoid(beta * logits)
    return loss.mean()

### 设置训练函数

In [10]:
from torch import nn
import time
iter_per_epoch = len(train_loader) # 计算每个 epoch 的迭代次数
def get_lr(current_step, total_steps, lr):
    # 余弦退火学习率调度
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

def train_epoch(epoch):
    start_time = time.time()
    for step, batch in enumerate(train_loader):
        # 提取数据
        x_chosen = batch['x_chosen'].to(train_args.device)
        x_rejected = batch['x_rejected'].to(train_args.device)
        y_chosen = batch['y_chosen'].to(train_args.device)
        y_rejected = batch['y_rejected'].to(train_args.device)
        mask_chosen = batch['mask_chosen'].to(train_args.device)
        mask_rejected = batch['mask_rejected'].to(train_args.device)
        # 正反例拼接
        x = torch.cat([x_chosen, x_rejected], dim=0)
        y = torch.cat([y_chosen, y_rejected], dim=0)
        mask = torch.cat([mask_chosen, mask_rejected], dim=0)

        lr = get_lr(epoch * iter_per_epoch + step, train_args.epochs * iter_per_epoch, train_args.learning_rate)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with ctx:
            with torch.no_grad(): # 计算 ref 模型输出
                ref_outputs = ref_model(x)
                ref_logits = ref_outputs.logits
            # print(f"res={res}")
            # print(f"X = {X}")
            ref_probs = logits_to_probs(ref_logits, y)
            ref_probs = ref_probs * mask # 得到 ref 概率
            outputs = model(x) # 计算 actor 模型输出
            logits = outputs.logits
            probs = logits_to_probs(logits, y)
            probs = probs * mask # 得到 actor 概率
            loss = dpo_loss(ref_probs, probs, beta=0.1) # dpo 损失
            loss = loss / train_args.accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % train_args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), train_args.grad_clip)

            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad(set_to_none=True)  # 清空梯度，为下一个iter做准备

        if step % train_args.log_interval == 0:
            spend_time = time.time() - start_time
            Logger(
                'Epoch:[{}/{}]({}/{}) loss:{:.3f} lr:{:.12f} epoch_Time:{}min:'.format(
                    epoch + 1,
                    train_args.epochs,
                    step,
                    iter_per_epoch,
                    loss.item() * train_args.accumulation_steps,
                    optimizer.param_groups[-1]['lr'],
                    spend_time / (step + 1) * iter_per_epoch // 60 - spend_time // 60))


        if (step + 1) % train_args.save_interval == 0:
            model.eval()
            moe_path = '_moe' if train_args.use_moe else ''
            ckp = f'{train_args.save_dir}/dpo_{config.embed_dim}{moe_path}.pth'
            Logger(f'保存模型到 {ckp}')
            state_dict = model.state_dict()

            state_dict = {k: v.half() for k, v in state_dict.items()}  # 半精度保存
            torch.save(state_dict, ckp)
            model.train()

### 训练一轮

In [11]:
iter_per_epoch = len(train_loader)
for epoch in range(train_args.epochs):
    train_epoch(epoch)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch:[1/1](0/1) loss:0.693 lr:0.000550000000 epoch_Time:0.0min:
